# Probabilistic Numerical Methods

Probabilistic numerical methods solve numerical tasks by treating them as problems of statistical inference. This approach captures uncertainty arising from finite computational resources and from stochastic input.

Suppose we are given the following problem.

In [1]:
# Make inline plots vector graphics instead of raster graphics
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

# Plotting
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [9.5, 6]
plt.rcParams['font.size'] = 18 
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsfonts}', r'\usepackage{amsmath}']

## A Simple Numerical Problem

## Univariate Random Variables

Instantiate and manipulate random variables and linear operators.

In [2]:
import numpy as np
import probnum as pn

# Random seed
np.random.seed(42)

# Gaussian random variable
X = RandomVariable(distribution=Normal(mean=0, cov=1))

# Arithmetic operations between scalars and random variables
Y = 2 * X - 3
print(Y)

<() RandomVariable with dtype=<class 'float'>>
